## 문자 문제

In [4]:
s = 'cAfé'

b = s.encode('utf8')
b, len(b)

(b'cAf\xc3\xa9', 5)

In [5]:
b.decode('utf8')

'cAfé'

In [12]:
b[1] # 16진수 41 --> 10진수 65

65

In [17]:
'€'.encode('utf-8'), 'é'.encode('utf-8'), 'A'.encode('utf-8')

(b'\xe2\x82\xac', b'\xc3\xa9', b'A')

In [12]:
'abcABC 가, 나, 다'.encode('utf-8')

b'abcABC \xea\xb0\x80, \xeb\x82\x98, \xeb\x8b\xa4'

## 바이트에 대한 기본 지식   
- 화면에 출력 가능한 아스키 문자는 아스키 문자 그대로 출력한다. (공백에서 ~까지)   
- 탭, 개행 문자, 캐리지 리턴?, 백슬래시는 이스케이프 시퀀스로 출력한다. (\t, \n, \r, \\)   
  - 캐리지 리턴 : 맨 앞으로 이동하기.   https://jw910911.tistory.com/90
- 그 외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력한다.


In [60]:
cafe = bytes('cAfé', encoding='utf-8')

cafe, cafe[1], cafe[:1]

(b'cAf\xc3\xa9', 65, b'c')

In [24]:
cafe_arr = bytearray(cafe)
cafe_arr, cafe_arr[-1:]

(bytearray(b'caf\xc3\xa9'), bytearray(b'\xa9'))

s[0] == s[:1]이 되는 시퀀스형은 str이 유일하다.   
실용적이긴 하지만 str의 이런 작동방식은 예외적인 것이다.   
그 외 모든 시퀀스의 경우, s[i]는 항목 하나를, s[i:i+1]은 안에 s[i]항목을 가진 동일한 자료형의 시퀀스를 반환한다.   

In [28]:
string = 'abced'

print(string[0] == string[:1])
print(string[0] is string[:1])
print(id(string[0]), id(string[:1]))

True
True
2798007328368 2798007328368


In [32]:
id('a')

2798007328368

In [51]:
lst = [1, 2, 3, 4, 5]

print(lst[0], lst[:1])
print(lst[0] == lst[:1])
print(id(lst[0]), id(lst[:1]))

1 [1]
False
2798003054896 2798087030464


In [58]:
id(1), id([1])

(2798003054896, 2798076964032)

In [61]:
# 공백으로 구분된 16진수 쌍 파싱하기
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

버퍼와 같은 객체로부터 bytes나 bytearray 객체를 생성하면 언제나 바이트를 **복사**한다.   
이와 반대로 memoryview는 이진 데이터 구조체 간에 메모리를 **공유**할 수 있게 해준다.

In [62]:
import array
# 'h' 타입코드는 short int (16비트) 형의 배열을 생성한다.
numbers = array.array('h', [-2, -1, 0, 1, 2])
# octets는 바이트 사본을 가지고 있다.
octets = bytes(numbers)
# 5개의 short int 형을 나타내는 10바이트이다.
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

In [16]:
import struct
fmt = '<3s3sHH'
with open('img.gif', 'rb') as fp:    
    img = memoryview(fp.read())  # 이 때 아무런 바이트도 복사하지 않는다.

In [17]:
header = img[:5]  # 이 때도 아무런 바이트도 복사하지 않는다.
bytes(header)  # 이 때는 복사한다. 20바이트가 복사된다.

b'GIF89'

In [18]:
struct.unpack(fmt, header)

error: unpack requires a buffer of 10 bytes

In [68]:
del header, img

## 기본 인코더/디코더

In [70]:
for codec in ['EUC-KR', 'utf_8', 'utf_16']:
    print(codec, 'El Ni 하이'.encode(codec), sep='t')

EUC-KRtb'El Ni \xc7\xcf\xc0\xcc'
utf_8tb'El Ni \xed\x95\x98\xec\x9d\xb4'
utf_16tb'\xff\xfeE\x00l\x00 \x00N\x00i\x00 \x00X\xd5t\xc7'


## 인코딩/디코딩 문제 이해하기

### UnicodeEncodeError 처리하기

In [5]:
city = 'São Paulo'
print(city.encode('utf_8'))
print(city.encode('utf_16'))
print(city.encode('iso8859_1'))
print(city.encode('cp437'))

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'


UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [6]:
print(city.encode('cp437', errors='ignore'))
print(city.encode('cp437', errors='replace'))
print(city.encode('cp437', errors='xmlcharrefreplace'))

b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


### UnicodeDecodeError 처리하기

In [7]:
octets = b'Montr\xe9al'
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7'))
print(octets.decode('koi8_r'))
print(octets.decode('utf_8'))


Montréal
Montrιal
MontrИal


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [8]:
octets.decode('utf_8', errors='replace')

'Montr�al'

코드 안의 비아스키 식별자

In [19]:
물음표, 느낌표, 마침표 = [], [], []

texts = '안녕하세요?|반갑습니다.|다음에 또 만나요!'.split('|')
for text in texts:
    for punc, lst in zip(['?', '!', '.'], (물음표, 느낌표, 마침표)):
        lst.append(text) if punc in text else None
        

물음표, 느낌표, 마침표

(['안녕하세요?'], ['다음에 또 만나요!'], ['반갑습니다.'])

### 유용한 깨진 문자

In [13]:
u16 = 'El Niño'.encode('utf_16')
print(u16)
list(u16)

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [1]:
u16le = 'El Niño'.encode('utf_16le')
print(list(u16le))
u16be = 'El Niño'.encode('utf_16be')
print(list(u16be))

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]


## 텍스트 파일 다루기

In [2]:
open('cafe.txt', 'w', encoding='utf-8').write('café')
open('cafe.txt', 'r').read()

'caf챕'

In [6]:
fp = open('cafe.txt', 'w', encoding='utf-8')
print(fp)
print(fp.write('café'))
fp.close()

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf-8'>
4


In [8]:
import os

print(os.stat('cafe.txt').st_size)
fp2 = open('cafe.txt')
print(fp2)
print(fp2.encoding)
fp2.read()



5
<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp949'>
cp949


'caf챕'

In [9]:
fp3 = open('cafe.txt', encoding='utf-8')
print(fp3)
fp3.read()

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf-8'>


'café'

In [10]:
fp4 = open('cafe.txt', 'rb')
print(fp4)
fp4.read()

<_io.BufferedReader name='cafe.txt'>


b'caf\xc3\xa9'

In [13]:
import sys, locale
expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'cp949'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'cp949'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


## 유니코드 정규화하기

In [17]:
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(len(s1), len(s2))
s1 == 2

café café
4 5


False

In [18]:
# 유니코드 정규화 함수

from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'

print(len(s1), len(s2))
print(len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)), len(normalize('NFD', s2)))
print(normalize('NFC', s1) == normalize('NFC', s2))
print(normalize('NFD', s1) == normalize('NFD', s2))

4 5
4 4
5 5
True
True


In [19]:
from unicodedata import normalize, name
ohm = '\u2126'
print(name(ohm))

ohm_c = normalize('NFC', ohm)
print(name(ohm_c))

print(ohm == ohm_c)
print(normalize('NFC', ohm) == normalize('NFC', ohm_c))


OHM SIGN
GREEK CAPITAL LETTER OMEGA
False
True


In [21]:
ohm, ohm_c

('Ω', 'Ω')

In [25]:
# 더욱 강력한 정규화 NFKC, NFKD

half = '½'
print(normalize('NFKC', half))
four_squared = '4²'
print(normalize('NFKC', four_squared))
micro = '\u00B5'
micro_kc = normalize('NFKC', micro)
print(micro, micro_kc)
print(ord(micro), ord(micro_kc))
print(name(micro), name(micro_kc))

1⁄2
42
µ μ
181 956
MICRO SIGN GREEK SMALL LETTER MU


### 케이스 폴딩
본질적으로 케이스 폴딩은 모든 텍스트를 소문자로 변환하는 연산이며, 약간의 변환을 동반한다.

In [28]:
micro_cf = micro.casefold()
print(name(micro_cf))
print(micro, micro_cf)
eszett = 'ß'
print(name(eszett))
eszett_cf = eszett.casefold()
print(eszett, eszett_cf)

GREEK SMALL LETTER MU
µ μ
LATIN SMALL LETTER SHARP S
ß ss


### 정규화된 텍스트 매칭을 위한 유틸리티 함수

In [30]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold())

In [31]:
print(s1, s2)
print(nfc_equal(s1, s2))
print(nfc_equal('a', 'A'))
s3 = 'Straße'
s4 = 'strasse'
print(nfc_equal(s3, s4))
print(fold_equal(s3, s4))
print(fold_equal(s1, s2))
print(fold_equal('A', 'a'))

café café
True
False
False
True
True
True


### 극단적인 정규화 : 발음 구별 기호 제거하기

In [32]:
import unicodedata
import string

def shave_marks(txt):
    """ 발음 구별 기호 모두 제거"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)
    

In [33]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
print(shave_marks(order))

greek = 'Ζέφυρος, Zéfiro'
print(shave_marks(greek)) 

“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”
Ζεφυρος, Zefiro


In [34]:
def shave_marks_latin(txt):
    """ 발음 구별 기호 모두 제거"""
    norm_txt = unicodedata.normalize('NFD', txt)
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue
        keepers.append(c)
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
        
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)
    

## 유니코드 텍스트 정렬하기
파이썬에서 비아스키 텍스트는 locale.strxfrm() 함수를 이용해 변환하는 것이 표준이다.

In [37]:
fruits = ['cafu', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'cafu', 'cajá']

In [39]:
import locale
locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')
sorted_fruits = sorted(fruits, key=locale.strxfrm)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cafu', 'cajá']

### 유니코드 대조 알고리즘을 이용한 정렬

In [41]:
import pyuca
coll = pyuca.Collator()
# 지역 정보가 필요 없다.
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cafu', 'cajá']

## 이중 모드 str 및 bytes API

In [44]:
import re

re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
# bytes에 정규식을 사용하면 아스키 범위를 벗어나는 문자들은 숫자나 단어로 처리하지 않는다.
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
            " as 1729 = 1³ + 12³ = 9³ + 10³.")

text_bytes = text_str.encode('utf_8')

print('Text', repr(text_str), sep='\n ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))
print('  bytes :', re_numbers_bytes.findall(text_bytes))
print('Words')
print('  str  :', re_words_str.findall(text_str))
print(' bytes :', re_words_bytes.findall(text_bytes))


Text
 'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes : [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
 bytes : [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


In [4]:
import os

print(os.listdir('.'))
print(os.listdir(b'.'))

['cafe.txt', 'Chapter4.ipynb', 'digit-of-삼.txt', 'dummy', 'img.gif']
[b'cafe.txt', b'Chapter4.ipynb', b'digit-of-\xec\x82\xbc.txt', b'dummy', b'img.gif']


In [5]:
korean_name_bytes = os.listdir(b'.')[2]
korean_name_str = korean_name_bytes.decode('ascii', 'surrogateescape')
# \x -> \udc
korean_name_str

'digit-of-\udcec\udc82\udcbc.txt'

In [6]:
korean_name_str.encode('ascii', 'surrogateescape')

b'digit-of-\xec\x82\xbc.txt'